In [ ]:
%reset

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from keras.utils import np_utils

from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/project/*Input Data Process')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('MSFT1.csv') # put the raw data downloaded from WRDS-TAQ-Quotes

In [ ]:
df.head() # see the data structure

,DATE,TIME_M,BID,BIDSIZ,ASK,ASKSIZ,SYM_ROOT,SYM_SUFFIX
0,01/11/2021,9:30:00.006921254,331.18,1,331.50,107,MSFT,NaN
1,01/11/2021,9:30:00.007295640,331.18,7,331.50,107,MSFT,NaN
2,01/11/2021,9:30:00.013154737,331.00,2,331.50,10,MSFT,NaN
3,01/11/2021,9:30:00.013989420,325.32,1,343.16,1,MSFT,NaN
4,01/11/2021,9:30:00.021584062,331.18,7,331.46,1,MSFT,NaN


In [ ]:
len(df)

4015942

In [ ]:
# create a new dataframe and clean the data (exclude data which equals to zero)
# exclude NA  and sample the data based on 10 seconds
df_time = df[(df['BID']!= 0) & (df['BIDSIZ']!= 0) & (df['ASK']!= 0) & (df['ASKSIZ']!= 0)]

df_time['TIME_M'] = df_time['TIME_M'].apply(lambda x: int(''.join(list(filter(lambda x: x.isdigit(), list(x)[0:8])))))

# here we also include quotes happened around exact 10 seconds to have enough order book depth
df_time = df_time[(df_time['TIME_M']%10 == 0)|(df_time['TIME_M']%10 == 1)|(df_time['TIME_M']%10 == 2)
                 |(df_time['TIME_M']%10 == 9)|(df_time['TIME_M']%10 == 8)].drop(['SYM_ROOT','SYM_SUFFIX'], axis = 1)

df_time['TIME_M'][(df_time['TIME_M']%10 == 1)|(df_time['TIME_M']%10 == 2)] = df_time['TIME_M'][(df_time['TIME_M']%10 == 1)|(df_time['TIME_M']%10 == 2)].apply(lambda x: 10*np.floor(x*0.1))
df_time['TIME_M'][(df_time['TIME_M']%10 == 9)|(df_time['TIME_M']%10 == 8)] = df_time['TIME_M'][(df_time['TIME_M']%10 == 9)|(df_time['TIME_M']%10 == 8)].apply(lambda x: 10*np.ceil(x*0.1))

df_time['TIME_M'][(df_time['TIME_M']-60)%100 == 0] = df_time['TIME_M'][(df_time['TIME_M']-60)%100 == 0] + 40

df_time['TIME_M'][(df_time['TIME_M']-6000)%10000 == 0] = df_time['TIME_M'][(df_time['TIME_M']-6000)%10000 == 0] + 4000

In [ ]:
# reorganize whole data structure
# !! cost a very very very long long long time here !!

df_bid = pd.DataFrame(columns = ['Date', 'Time', 'BID', 'BIDSIZ'])
df_ask = pd.DataFrame(columns = ['Date', 'Time', 'ASK', 'ASKSIZ'])

for date in tqdm(np.unique(df_time['DATE'])):
  for t in tqdm(np.unique(df_time['TIME_M'])):
    sequence = df_time[(df_time['DATE'] == date) & (df_time['TIME_M'] == t)]

    # deal with bid part
    # sort the price from low to high
    sequence = sequence.sort_values(by = ['BID'])
    if len(np.unique(sequence['BID'])) < 10:
      continue

    # calculate the bid volume of each price at the Date 'date' and Time 't'
    bidsiz = []
    for bid in np.unique(sequence['BID'])[-10:]:
      bidsiz.append(sum(sequence['BIDSIZ'][sequence['BID'] == bid]))

    # deal with ask part
    sequence = sequence.sort_values(by = ['ASK'])
    if len(np.unique(sequence['ASK'])) < 10:
      continue

    asksiz = []
    for ask in np.unique(sequence['ASK'])[:10]:
      asksiz.append(sum(sequence['ASKSIZ'][sequence['ASK'] == ask]))

    # append the bid price and corresbonding bid volume with index 'Date' and 'Time'
    df_bid = df_bid.append({'Date': date, 'Time': t, 'BID':np.unique(sequence['BID'])[-10:], 'BIDSIZ':bidsiz}, ignore_index=True)
    df_ask = df_ask.append({'Date':date, 'Time': t, 'ASK':np.unique(sequence['ASK'])[:10], 'ASKSIZ':asksiz}, ignore_index=True)

In [ ]:
df_bid

,Date,Time,BID,BIDSIZ
0,01/11/2021,93000,"[331.36, 331.37, 331.38, 331.39, 331.4, 331.41...","[432, 349, 161, 74, 203, 81, 145, 40, 48, 1]"
1,01/11/2021,93010,"[331.31, 331.32, 331.33, 331.34, 331.35, 331.3...","[47, 77, 81, 148, 98, 24, 4, 42, 19, 17]"
2,01/11/2021,93020,"[331.14, 331.15, 331.16, 331.17, 331.18, 331.1...","[116, 32, 40, 20, 17, 12, 7, 6, 8, 8]"
3,01/11/2021,93030,"[330.92, 330.93, 330.94, 330.95, 330.96, 330.9...","[66, 105, 108, 69, 30, 42, 33, 2, 7, 7]"
4,01/11/2021,93040,"[330.85, 330.86, 330.87, 330.88, 330.89, 330.9...","[535, 848, 640, 222, 428, 94, 73, 13, 77, 17]"
...,...,...,...,...
2127,01/11/2021,155920,"[329.52, 329.53, 329.54, 329.55, 329.56, 329.5...","[59, 742, 928, 198, 202, 94, 258, 29, 32, 5]"
2128,01/11/2021,155930,"[329.5, 329.51, 329.52, 329.53, 329.54, 329.55...","[37, 29, 54, 523, 1028, 194, 309, 363, 160, 90]"
2129,01/11/2021,155940,"[329.59, 329.6, 329.61, 329.62, 329.63, 329.64...","[1665, 386, 380, 79, 130, 231, 270, 327, 115, 11]"
2130,01/11/2021,155950,"[330.06, 330.07, 330.08, 330.09, 330.1, 330.11...","[84, 44, 157, 99, 138, 97, 45, 93, 100, 41]"


In [ ]:
# reorganize the data struture on each timestamp

# ASK side

ASK = pd.DataFrame(columns = ['PRICE_ASK_0','PRICE_ASK_1','PRICE_ASK_2','PRICE_ASK_3','PRICE_ASK_4',
                              'PRICE_ASK_5','PRICE_ASK_6','PRICE_ASK_7','PRICE_ASK_8','PRICE_ASK_9'])
                                 
ASKSIZ = pd.DataFrame(columns = ['VOLUME_ASK_0','VOLUME_ASK_1','VOLUME_ASK_2','VOLUME_ASK_3','VOLUME_ASK_4',
                                 'VOLUME_ASK_5','VOLUME_ASK_6','VOLUME_ASK_7','VOLUME_ASK_8','VOLUME_ASK_9'])

for j in range(len(df_ask)):
  ASK = ASK.append(pd.DataFrame(df_ask['ASK'][j].reshape(1,-1), columns = ASK.columns), ignore_index = True)
  ASKSIZ = ASKSIZ.append(pd.DataFrame(np.asarray(df_ask['ASKSIZ'][j]).reshape(1,-1), columns = ASKSIZ.columns), ignore_index = True)

ASK = pd.concat([ASK, ASKSIZ], axis = 1)

# BID SIDE

BID = pd.DataFrame(columns = ['PRICE_BID_0','PRICE_BID_1','PRICE_BID_2','PRICE_BID_3','PRICE_BID_4',
                              'PRICE_BID_5','PRICE_BID_6','PRICE_BID_7','PRICE_BID_8','PRICE_BID_9'])
                                 
BIDSIZ = pd.DataFrame(columns = ['VOLUME_BID_0','VOLUME_BID_1','VOLUME_BID_2','VOLUME_BID_3','VOLUME_BID_4',
                                 'VOLUME_BID_5','VOLUME_BID_6','VOLUME_BID_7','VOLUME_BID_8','VOLUME_BID_9'])

for j in range(len(df_bid)):
  BID = BID.append(pd.DataFrame(df_bid['BID'][j].reshape(1,-1), columns = BID.columns), ignore_index = True)
  BIDSIZ = BIDSIZ.append(pd.DataFrame(np.asarray(df_bid['BIDSIZ'][j]).reshape(1,-1), columns = BIDSIZ.columns), ignore_index = True)


BID.insert(0,'Time',df_bid['Time'])
BID.insert(0,'Date',df_bid['Date'])
BID = pd.concat([BID, BIDSIZ], axis = 1)

Data = pd.concat([BID, ASK], axis = 1)

# -------------------------------------
# Notice: "TWTR" is Stock '1', "AAPL" is Stock '2', "F" is Stock '3'
# Notice: "MSFT" is Stock '1', "BILI" is Stock '2', "AAPL" is Stock '3'
# -------------------------------------
Data.insert(0,'Stock',1)

In [ ]:
# reorganize the bid side to make the best bid price as the highest bid price instead of lowest one
Data = Data.rename(columns={'PRICE_BID_0':'PRICE_BID_9', 'PRICE_BID_1':'PRICE_BID_8', 'PRICE_BID_2':'PRICE_BID_7',
                     'PRICE_BID_3':'PRICE_BID_6', 'PRICE_BID_4':'PRICE_BID_5', 'PRICE_BID_5':'PRICE_BID_4',
                     'PRICE_BID_6':'PRICE_BID_3', 'PRICE_BID_7':'PRICE_BID_2', 'PRICE_BID_8':'PRICE_BID_1',
                     'PRICE_BID_9':'PRICE_BID_0', 'VOLUME_BID_0':'VOLUME_BID_9','VOLUME_BID_1':'VOLUME_BID_8',
                     'VOLUME_BID_2':'VOLUME_BID_7', 'VOLUME_BID_3':'VOLUME_BID_6','VOLUME_BID_4':'VOLUME_BID_5',
                     'VOLUME_BID_5':'VOLUME_BID_4','VOLUME_BID_6':'VOLUME_BID_3','VOLUME_BID_7':'VOLUME_BID_2',
                     'VOLUME_BID_8':'VOLUME_BID_1','VOLUME_BID_9':'VOLUME_BID_0'})

# save the reorganized data
Data.to_csv('Data.csv')

In [4]:
# Example of the output form
AAPL = pd.read_csv('AAPL.csv')
print(AAPL.columns)
AAPL.head()

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Stock', 'Date', 'Time', 'PRICE_BID_9',
       'PRICE_BID_8', 'PRICE_BID_7', 'PRICE_BID_6', 'PRICE_BID_5',
       'PRICE_BID_4', 'PRICE_BID_3', 'PRICE_BID_2', 'PRICE_BID_1',
       'PRICE_BID_0', 'VOLUME_BID_9', 'VOLUME_BID_8', 'VOLUME_BID_7',
       'VOLUME_BID_6', 'VOLUME_BID_5', 'VOLUME_BID_4', 'VOLUME_BID_3',
       'VOLUME_BID_2', 'VOLUME_BID_1', 'VOLUME_BID_0', 'Date.1', 'Time.1',
       'PRICE_ASK_0', 'PRICE_ASK_1', 'PRICE_ASK_2', 'PRICE_ASK_3',
       'PRICE_ASK_4', 'PRICE_ASK_5', 'PRICE_ASK_6', 'PRICE_ASK_7',
       'PRICE_ASK_8', 'PRICE_ASK_9', 'VOLUME_ASK_0', 'VOLUME_ASK_1',
       'VOLUME_ASK_2', 'VOLUME_ASK_3', 'VOLUME_ASK_4', 'VOLUME_ASK_5',
       'VOLUME_ASK_6', 'VOLUME_ASK_7', 'VOLUME_ASK_8', 'VOLUME_ASK_9'],
      dtype='object')


,Unnamed: 0,Unnamed: 0.1,Stock,Date,Time,PRICE_BID_9,PRICE_BID_8,PRICE_BID_7,PRICE_BID_6,PRICE_BID_5,...,VOLUME_ASK_0,VOLUME_ASK_1,VOLUME_ASK_2,VOLUME_ASK_3,VOLUME_ASK_4,VOLUME_ASK_5,VOLUME_ASK_6,VOLUME_ASK_7,VOLUME_ASK_8,VOLUME_ASK_9
0,0,0,3,01/11/2021,93000,148.91,148.92,148.93,148.94,148.95,...,1,2,248,184,15,434,79,1042,86,740
1,1,1,3,01/11/2021,93010,149.36,149.37,149.38,149.39,149.40,...,35,84,309,374,446,208,109,88,406,717
2,2,2,3,01/11/2021,93020,149.53,149.54,149.55,149.56,149.57,...,116,35,359,249,510,1022,906,570,453,294
3,3,3,3,01/11/2021,93030,149.34,149.35,149.36,149.37,149.38,...,35,174,626,1470,1330,2388,2308,1427,1578,2101
4,4,4,3,01/11/2021,93040,149.31,149.32,149.33,149.34,149.35,...,70,647,1308,2161,3019,5443,1853,732,288,828
